# TP1
## Question 2: ZLW

In [2]:
import numpy as np

### File loading and dictionary initialization

In [3]:
FILES = [f"texte_{i}.txt" for i in range(1,6)]
PATH = "../data/textes/"

def load_file(index: int) -> str:
	if index < 0 or index >= len(FILES):
		raise ValueError("Index out of range")
	with open(PATH+FILES[index],"r") as f:
		msg = f.read()
	return msg

def init_dict(msg: str) -> dict[str, str]:
	dict_symb = {}
	n_symb = 0
	for i in range(len(msg)):
		if msg[i] not in dict_symb:
			dict_symb[msg[i]] = f"{n_symb:b}"
			n_symb +=1
	return dict_symb

### Utils

In [4]:
def dict_update_size(dict_symb: dict[str, str]):
	n_symb = len(dict_symb)
	for k, v in dict_symb.items():
		dict_symb[k] = v.zfill(int(np.ceil(np.log2(n_symb))))

log_size = lambda x: np.ceil(np.log2(x))


### Compression algorithm

In [5]:
def compress(msg: str, dict_symb: dict[str, str]) -> (list[str], int):
	i, length = 0, 0
	n_symb = len(dict_symb)
	coded_msg = []
	while i < len(msg):
		# Next coded string
		next_str = msg[i]
		# Same, but with extra character (for the dictionary)
		next_str_extra = msg[i]

		# Tries to fit the largest string possible in the dictionary
		while next_str_extra in dict_symb and i < len(msg):
			i += 1
			next_str = next_str_extra
			if i < len(msg): # If there is still characters to read
				next_str_extra += msg[i]

		# Coding of the string
		bin_code = dict_symb[next_str]
		coded_msg.append(bin_code)
		length += len(bin_code)
		# Adding the new string to the dictionary
		if i < len(msg):
			dict_symb[next_str_extra] = f"{n_symb:b}"
			n_symb += 1

		# Updating symbols size if necessary
		if log_size(n_symb) > len(coded_msg[-1]):
			dict_update_size(dict_symb)
	return coded_msg, length

In [7]:
import time
def run(index: int, verbose: bool = True):
	start = time.time()
	msg = load_file(index)
	loaded = time.time()
	if verbose:
		print(f"Message: {msg}")
	dict_symb = init_dict(msg)
	initial_length = int(log_size(len(dict_symb))*len(msg))
	dict_update_size(dict_symb)
	dict_loaded = time.time()
	if verbose:
		print(f"Binary symbols: {dict_symb}")
		print(f"Initial length: {initial_length}")
	coded_msg, length = compress(msg, dict_symb)
	compressed = time.time()
	print(f"Coded message: {coded_msg}")
	print(f"Or : {''.join(coded_msg)}")

	if verbose:
		print("")
		print(f"Length = {length}")
		print(f"Original length = {initial_length}")
		print(f"Compression rate = {100 - length/initial_length*100:.2f}%")
		print(f"Compression factor = {initial_length/length:.2f}")

	if verbose:
		print("")
		print(f"Loading time: {(loaded-start)*10**3:.2f}ms")
		print(f"Dictionary loading time: {(dict_loaded-loaded)*10**3:.2f}ms")
		print(f"Compression time: {(compressed-dict_loaded)*10**3:.2f}ms")
		print(f"Total time: {(compressed-start)*10**3:.2f}ms")
run(2)


Message: FOFOFOEOKOWKOOWONOOOOOOWOENFOEOOOJSODNOJFSOJKOOOOBV
Binary symbols: {'F': '0000', 'O': '0001', 'E': '0010', 'K': '0011', 'W': '0100', 'N': '0101', 'J': '0110', 'S': '0111', 'D': '1000', 'B': '1001', 'V': '1010'}
Initial length: 204
Coded message: ['0000', '0001', '1011', '1011', '0010', '0001', '00011', '00001', '00100', '10001', '10010', '00001', '00101', '00001', '11000', '11001', '00100', '00001', '00010', '00101', '01110', '11001', '000110', '000111', '000001', '001000', '010111', '000110', '000000', '100010', '000110', '010100', '011000', '001001', '001010']
Or : 00000001101110110010000100011000010010010001100100000100101000011100011001001000000100010001010111011001000110000111000001001000010111000110000000100010000110010100011000001001001010

Length = 182
Original length = 204
Compression rate = 10.78%
Compression factor = 1.12

Loading time: 1.60ms
Dictionary loading time: 0.69ms
Compression time: 0.65ms
Total time: 2.94ms
